## overview

In [1]:
# default package
import logging
import sys 
import os
import pathlib
import IPython
import random

In [2]:
# third party package
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import psycopg2
import sqlalchemy
from sqlalchemy.sql.expression import column,select,table
from sqlalchemy import create_engine
import sklearn.datasets

In [3]:
# my package
sys.path.append(os.path.join(pathlib.Path().resolve(),"../"))

In [4]:
# reload settings
%load_ext autoreload
%autoreload 2

In [5]:
# logger
logger=logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [6]:
## grap setting
sns.set()

In [7]:
## db setting
engine = create_engine('postgresql://postgres:postgres@db:5432/postgres')
with engine.connect() as conn:
    print(conn.closed)

False


## sql to python

In [8]:
def ct(file:str,chap_num:int,num:int,sub_num:int=None,engine:sqlalchemy.engine=engine)->None:
    """
    pubulic schema内のtableをすべて削除してから、指定のtable作成
    """
    if sub_num==None: 
        filename=f"../code-ssql2/ddl/ssql2-{file}{chap_num:02}_{num:02}"
    else:
        filename=f"../code-ssql2/ddl/ssql2-{file}{chap_num:02}_{num:02}_{sub_num}"
        
    with open(filename,"r") as f:
        stmt=f.read() 
    with engine.connect() as conn:
        conn.execute("drop schema public cascade; create schema public;")
        conn.execute(stmt)

In [9]:
def exe(stmt:str,engine:sqlalchemy.engine=engine):
    """
    sql実行
    """
    with engine.connect() as conn:
        query=conn.execute(stmt)
        try:
            result=query.fetchall()
            return result
        except:
            return None

In [10]:
def exe_df(stmt:str,engine:sqlalchemy.engine=engine)->pd.DataFrame:
    """
    sql(select)を実行してdataframeを返す
    """
    with engine.connect() as conn:
        df=pd.read_sql(stmt,conn)
    return df 

## List

In [11]:
ct("List",5,1)
exe_df("""
select 出金額,出金額+100,'SQL'
from 家計簿
""")

,出金額,?column?,?column?
0,380,480,SQL
1,0,100,SQL
2,2800,2900,SQL
3,5000,5100,SQL
4,7560,7660,SQL


In [12]:
ct("List",5,2)
exe_df("""
select 出金額,出金額+100 as increase
from 家計簿
""")

,出金額,increase
0,380,480
1,0,100
2,2800,2900
3,5000,5100
4,7560,7660


In [13]:
ct("List",5,3)
exe("""
insert into 家計簿
(出金額)
values (1000+105)
""")
exe("select * from 家計簿")

[(datetime.date(2018, 2, 3), '食費', 'コーヒーを購入', 0, 380),
 (datetime.date(2018, 2, 10), '給料', '1月の給料', 280000, 0),
 (datetime.date(2018, 2, 11), '教養娯楽費', '書籍を購入', 0, 2800),
 (datetime.date(2018, 2, 14), '交際費', '同期会の会費', 0, 5000),
 (datetime.date(2018, 2, 18), '水道光熱費', '1月の電気代', 0, 7560),
 (None, None, None, None, 1105)]

In [14]:
ct("List",5,4)
exe("""
update 家計簿
set 出金額=出金額+100
""")
exe("select * from 家計簿")

[(datetime.date(2018, 2, 3), '食費', 'コーヒーを購入', 0, 480),
 (datetime.date(2018, 2, 10), '給料', '1月の給料', 280000, 100),
 (datetime.date(2018, 2, 11), '教養娯楽費', '書籍を購入', 0, 2900),
 (datetime.date(2018, 2, 14), '交際費', '同期会の会費', 0, 5100),
 (datetime.date(2018, 2, 18), '水道光熱費', '1月の電気代', 0, 7660)]

In [15]:
ct("List",5,5)
exe("""
select 費目,出金額,
case 費目 when '居住費' then '固定費'
when '水道光熱費' then '固定費'
else '変動費'
end as 出費の分類
from 家計簿 where 出金額>0
""")

[('食費', 380, '変動費'),
 ('教養娯楽費', 2800, '変動費'),
 ('交際費', 5000, '変動費'),
 ('水道光熱費', 7560, '固定費')]

In [16]:
ct("List",5,6)
exe("""
select 費目,入金額,
case when 入金額<5000 then 'お小遣い'
when 入金額<100000 then '一時収入'
when 入金額<300000 then '給料'
else '想定外'
end as 収入の分類
from 家計簿
where 入金額>0
""")

[('給料', 280000, '給料')]

In [17]:
ct("List",5,7)
exe_df("""
select メモ,length(メモ) as メモの長さ
from 家計簿
""")

,メモ,メモの長さ
0,コーヒーを購入,7
1,1月の給料,5
2,書籍を購入,5
3,同期会の会費,6
4,1月の電気代,6


In [18]:
ct("List",5,8)
exe_df("""
select メモ,length(メモ) as メモの長さ
from 家計簿
where length(メモ)<=10
""")

,メモ,メモの長さ
0,コーヒーを購入,7
1,1月の給料,5
2,書籍を購入,5
3,同期会の会費,6
4,1月の電気代,6


In [19]:
ct("List",5,9)
exe("""
select メモ,trim(メモ) 
from 家計簿
""")

[('コーヒーを購入', 'コーヒーを購入'),
 ('1月の給料', '1月の給料'),
 ('書籍を購入', '書籍を購入'),
 ('同期会の会費', '同期会の会費'),
 ('1月の電気代', '1月の電気代')]

In [20]:
ct("List",5,10)
exe("""
update 家計簿
set メモ=replace(メモ,'購入','買った')
""")
exe("select * from 家計簿")

[(datetime.date(2018, 2, 3), '食費', 'コーヒーを買った', 0, 380),
 (datetime.date(2018, 2, 10), '給料', '1月の給料', 280000, 0),
 (datetime.date(2018, 2, 11), '教養娯楽費', '書籍を買った', 0, 2800),
 (datetime.date(2018, 2, 14), '交際費', '同期会の会費', 0, 5000),
 (datetime.date(2018, 2, 18), '水道光熱費', '1月の電気代', 0, 7560)]

In [21]:
ct("List",5,11)
exe("""
select * from 家計簿
where substring(費目,1,3) like '%%費%%'
""")

[(datetime.date(2018, 2, 3), '食費', 'コーヒーを購入', 0, 380),
 (datetime.date(2018, 2, 14), '交際費', '同期会の会費', 0, 5000)]

In [22]:
ct("List",5,12)
exe("""
select concat(費目,':',メモ) from 家計簿
""")

[('食費:コーヒーを購入',),
 ('給料:1月の給料',),
 ('教養娯楽費:書籍を購入',),
 ('交際費:同期会の会費',),
 ('水道光熱費:1月の電気代',)]

In [23]:
ct("List",5,13)
exe("""
select 出金額,round(出金額,-2) 
from 家計簿
""")

[(380, Decimal('400')),
 (0, Decimal('0')),
 (2800, Decimal('2800')),
 (5000, Decimal('5000')),
 (7560, Decimal('7600'))]

In [24]:
ct("List",5,14)
exe("""
insert into 家計簿
values (current_date,'食費','ドーナツを買った',0,260)
""")
exe("select * from 家計簿")

[(datetime.date(2018, 2, 3), '食費', 'コーヒーを購入', 0, 380),
 (datetime.date(2018, 2, 10), '給料', '1月の給料', 280000, 0),
 (datetime.date(2018, 2, 11), '教養娯楽費', '書籍を購入', 0, 2800),
 (datetime.date(2018, 2, 14), '交際費', '同期会の会費', 0, 5000),
 (datetime.date(2018, 2, 18), '水道光熱費', '1月の電気代', 0, 7560),
 (datetime.date(2021, 2, 17), '食費', 'ドーナツを買った', 0, 260)]

In [25]:
ct("List",5,16)
exe("""
select coalesce(メモ,'メモはnullです')
from 家計簿
""")

[('自分へのご褒美',), ('メモはnullです',), ('メモはnullです',)]

## Q

In [26]:
ct("Q",5,1,"1_1")
exe("""
update 試験結果 set 午後1=平均点*4-午前-午後2-論述
where 受験者ID='SW1046'
""")
exe("select * from 試験結果")

[('SW1350', 65, 53, 70, None, 68),
 ('SW1877', None, 59, 56, 36, 56),
 ('SW1046', 86, 75, 68, 91, 80)]

In [27]:
ct("Q",5,1,2)
exe("""
select 受験者ID from 試験結果
where 午前>=60
and (午後1+午後2)>=120
and 論述>=平均点*4*0.3
""")

[('SW1350',)]

In [28]:
ct("Q",5,2,1)
exe("""
select 
case substring(trim(メールアドレス),length(trim(メールアドレス))-1,2)
when 'jp' then '日本'
else 'その他'
end
from 回答者
""")

[('日本',), ('その他',), ('その他',), ('その他',), ('その他',)]

In [29]:
ct("Q",5,2,2)
exe("""
select trim(メールアドレス),
concat(年齢,':',
case when 性別='M' then '男性'
    else  '女性'
    end
) as 属性
from 回答者
""")

[('suzuki.takashi@mailsample.jp', '51:男性'),
 ('philip@mailsample.uk', '26:男性'),
 ('hao@mailsample.cn', '35:男性'),
 ('marie@mailsample.fr', '43:女性'),
 ('hoa@mailsample.vn', '22:女性')]